In [1]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

tokenizer = AutoTokenizer.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")

c:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
csv_path = r'C:\Users\hp\Desktop\preply\lars\project\module 5\consolidated_data.csv'
df = pd.read_csv(csv_path)

In [3]:
df.head()

,Name,Content
0,EQUITY Factsheet_ A.P. Moller - Maersk A_S,"['maersk', 'ebitda', 'billion', 'consensu', 'e..."
1,EQUITY Factsheet_ Abbott Laboratories,"['expect', 'total', 'sale', 'contract', 'billi..."
2,EQUITY Factsheet_ AbbVie Inc.,"['expect', 'revenu', 'growth', 'billion', 'dec..."
3,EQUITY Factsheet_ Adobe Inc. (1),"['project', 'sale', 'grow', 'fy', 'fy', 'compa..."
4,EQUITY Factsheet_ Advanced Energy Industries Inc.,"['tight', 'suppli', 'condit', 'hurt', 'aei', '..."


In [4]:
def chunk_size(input_ids, attention_mask, total_len):
    prob_list = []
    start = 0
    win_length = 510
    loop = True

    while loop:
        end = start + win_length

        if end >= total_len:
            loop = False 
            end = total_len

        # Defining the text chunk:
        input_ids_chunk = input_ids[start:end]
        attention_mask_chunk = attention_mask[start:end]

        # Referring to specific classes:
        input_ids_chunk = [101] + input_ids_chunk + [102]
        attention_mask_chunk = [1] + attention_mask_chunk + [1]

        # Convert to PyTorch Tensors:
        input_dict = {'input_ids' : torch.Tensor([input_ids_chunk]).long(),
                      'attention_mask' : torch.Tensor([attention_mask_chunk]).int()}
        
        outputs = model(**input_dict)
        probabilities = torch.nn.functional.softmax(outputs[0], dim = -1)
        prob_list.append(probabilities)

        start = end

    return prob_list

In [5]:
def get_mean_from_prob(prob_list):
    with torch.no_grad():
        stacks = torch.stack(prob_list)

        stacks = stacks.resize(stacks.shape[0], stacks.shape[2])

        mean = stacks.mean(dim=0)

    return mean

In [16]:
text = df['Content'][5]
tokens = tokenizer.encode_plus(text, add_special_tokens=False)
input_ids = tokens['input_ids']
attention_mask = tokens['attention_mask']
total_len = len(input_ids)
prob_list = chunk_size(input_ids, attention_mask, total_len)
stacks = torch.stack(prob_list)
shape = stacks.shape
mean = get_mean_from_prob(prob_list)
print(torch.argmax(mean).item())

1


In [6]:
for text in df['Content']:
    tokens = tokenizer.encode_plus(text, add_special_tokens=False)
    input_ids = tokens['input_ids']
    attention_mask = tokens['attention_mask']
    total_len = len(input_ids)
    prob_list = chunk_size(input_ids, attention_mask, total_len)
    stacks = torch.stack(prob_list)
    shape = stacks.shape
    mean = get_mean_from_prob(prob_list)
    sentiment_value = torch.argmax(mean).item()

    if sentiment_value == 0:
        df['Sentiment'] = 'Negative'
    elif sentiment_value == 1:
        df['Sentiment'] = 'Neutral'
    else:
        df['Sentiment'] = 'Positive'

Token indices sequence length is longer than the specified maximum sequence length for this model (1699 > 512). Running this sequence through the model will result in indexing errors
c:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\_tensor.py:868: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\_tensor.py:868: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


In [7]:
df.head()

,Name,Content,Sentiment
0,EQUITY Factsheet_ A.P. Moller - Maersk A_S,"['maersk', 'ebitda', 'billion', 'consensu', 'e...",Neutral
1,EQUITY Factsheet_ Abbott Laboratories,"['expect', 'total', 'sale', 'contract', 'billi...",Neutral
2,EQUITY Factsheet_ AbbVie Inc.,"['expect', 'revenu', 'growth', 'billion', 'dec...",Neutral
3,EQUITY Factsheet_ Adobe Inc. (1),"['project', 'sale', 'grow', 'fy', 'fy', 'compa...",Neutral
4,EQUITY Factsheet_ Advanced Energy Industries Inc.,"['tight', 'suppli', 'condit', 'hurt', 'aei', '...",Neutral


In [9]:
df.to_csv(r'C:\Users\hp\Desktop\preply\lars\project\module 5\final_csv.csv')

print('Final CSV stored to Module 5.')

Final CSV stored to Module 5.
